# Linear regression
The linear regression formula we're going to use is
$$y_i = a_0 f_0(x_i) + a_1 f_1(x_i) + ... + a_n f_n(x_i)$$
and we're going to minimize Mean Squared Error:
$$MSE = \sum_i \left(\sum_j \left(a_j f_j(x_i)\right) - y_i\right)^2$$

In [2]:
#i "https://www.myget.org/F/angourimath/api/v3/index.json"
#r "nuget:AngouriMath.Interactive, 0.0.0-master-1646682623-e8428b2"

Restore sources https://www.myget.org/F/angourimath/api/v3/index.json Installed Packages AngouriMath.Interactive, 0.0.0-master-1646682623-e8428b2

Loading extensions from `AngouriMath.Interactive.dll`

LaTeX renderer binded. Enjoy!


In [3]:
open AngouriMath.FSharp.Shortcuts
open AngouriMath
open AngouriMath.FSharp.Functions
open AngouriMath.FSharp.Core
open AngouriMath.FSharp.Matrices

## Data

In [4]:
let X, Y = List.unzip [
    1., 1.
    2., 3.
    6., 5.
    8., 9.5
]
let XY = List.zip X Y

## Column transformations
These are functions of `x`. The linear regression formula we're going to use is
$$y_i = a_0 f_0(x_i) + a_1 f_1(x_i) + ... + a_n f_n(x_i)$$
and we're going to minimize Mean Squared Error:
$$MSE = \sum_i \left(\sum_j \left(a_j f_j(x_i)\right) - y_i\right)^2$$

In [5]:
let fs = [
    "1"
    "x"
]

In [6]:
// just summing all elements
let listAdd (a : Entity list) = List.fold (fun acc value -> acc + value) (parsed 0) a

## Solve system
To be able to minimize MSE, we shall say that
$$\forall k: \frac{\delta}{\delta a_k} MSE = 0$$
$$\forall k: \frac{\delta}{\delta a_k} \sum_i \left(\sum_j \left(a_j f_j(x_i)\right) - y_i\right)^2 = 0$$
$$\forall k: \sum_j (a_i \sum_j f_k(x_i) f_j(x_i)) - \sum_i y_i f_k(x_i) = 0$$
Now we can rewrite it into matrix:
$$
\begin{bmatrix}
\sum f_1(x_i) f_1(x_i) & \sum f_1(x_i) f_2(x_i) & ... & \sum f_1(x_i) f_n(x_i)\\
\sum f_2(x_i) f_1(x_i) & \sum f_2(x_i) f_2(x_i) & ... & \sum f_2(x_i) f_n(x_i)\\
\vdots           & \vdots                    &     & \vdots  \\
\sum f_n(x_i) f_1(x_i) & \sum f_n(x_i) f_2(x_i) & ... & \sum f_n(x_i) f_n(x_i)\\
\end{bmatrix}
\begin{bmatrix}
a_1\\
a_2\\
\vdots\\
a_n\\
\end{bmatrix}
=
\begin{bmatrix}
\sum y_i f_1(x_i)\\
\sum y_i f_2(x_i)\\
\vdots\\
\sum y_i f_n(x_i)\\
\end{bmatrix}
$$

In [7]:
let A = matrixWith (Seq.length fs) (Seq.length fs + 1) (fun x y ->
    if y = Seq.length fs then
        XY |> List.map (fun (xi, yi) -> parsed yi * (fs[x] <|- ("x", xi))) |> listAdd
    else
        X |> List.map (fun xi -> (fs[x] <|- ("x", xi)) * (fs[y] <|- ("x", xi))) |> listAdd
)
let Ac = simplified A :?> Entity.Matrix
let rref = Ac.ReducedRowEchelonForm
rref

In [8]:
let aCoefs = [ 0 .. Seq.length fs - 1] |> List.map (fun i -> rref.T[Seq.length fs, i])
aCoefs

index,value
0,\[\frac{43}{262}\]
1,\[\frac{275}{262}\]


In [9]:
let FA = List.zip fs aCoefs

## Compute the estimated
As data now we take `x` between the max and min, and estimate corresponding `y`.

In [10]:
let xMin, xMax = List.max X, List.min X
let xEst = [ xMin .. (xMax - xMin) / 100. .. xMax ]
let yEst = xEst |> List.map (fun x -> FA |> List.map (fun (f, a) -> (f <|- ("x", x)) * a) |> listAdd) |> List.map (fun x -> x.EvalNumerical() |> double)

## Visualize
Display the initial data and our estimator

In [11]:
open Plotly.NET
let chart1 = Chart.Point(X, Y)
let chart2 = Chart.Point(xEst, yEst)
Chart.combine [chart1; chart2]

<!-- Plotly chart will be drawn inside this DIV -->

# Build estimator
Function which, for given data and transformations, returns estimator

In [12]:
let estimator (X : float list) (Y : float list) (fs : string list) : float -> float =
    let XY = List.zip X Y
    let A = matrixWith (Seq.length fs) (Seq.length fs + 1) (fun x y ->
        if y = Seq.length fs then
            XY |> List.map (fun (xi, yi) -> parsed yi * (fs[x] <|- ("x", xi))) |> listAdd
        else
            X |> List.map (fun xi -> (fs[x] <|- ("x", xi)) * (fs[y] <|- ("x", xi))) |> listAdd
    )
    let Ac = simplified A :?> Entity.Matrix
    let rref = Ac.ReducedRowEchelonForm
    let aCoefs = [ 0 .. Seq.length fs - 1] |> List.map (fun i -> rref.T[Seq.length fs, i])
    printfn $"Found coefs: {aCoefs}"
    let FA = List.zip fs aCoefs
    let estimator (x : float) : float =
        FA |> List.map (fun (f, a) -> (f <|- ("x", x)) * a) |> listAdd
        |> (fun x -> x.EvalNumerical())
        |> double
    let yEst : float list = X |> List.map estimator
    let MSE = (List.zip Y yEst |> List.map (fun (y, yEst) -> (y - yEst) * (y - yEst)) |> List.sum) / (Seq.length X |> double)
    printfn $"MSE: {MSE}"
    estimator

### Try what we already did

In [13]:
let trainX, trainY = List.unzip [
    1., 1.
    2., 3.
    6., 5.
    8., 9.5
]
let est = estimator trainX trainY [ "1"; "x" ]

Found coefs: [43/262; 275/262]
MSE: 0.9017175572519089


In [14]:
let testX = [ 0.5 .. 0.05 .. 10. ]
let estY = testX |> List.map est

In [15]:
Chart.combine [
    Chart.Point(trainX, trainY)
    Chart.Point(testX, estY)
]

<!-- Plotly chart will be drawn inside this DIV -->

### Add data, complicate the transformations

In [16]:
let train2X, train2Y = List.unzip [
    1., 1.
    2., 3.
    2., 3.5
    6., 5.
    7., 2.
    8., 9.5
    9., 6.
]
let est2 = estimator trainX trainY [ "1"; "x"; "x^3" ]

Found coefs: [2885/1994; 247/997; 23/1994]
MSE: 0.4096038114343028


In [17]:
let test2X = [ 0.5 .. 0.05 .. 10. ]
let est2Y = test2X |> List.map est2

In [18]:
Chart.combine [
    Chart.Point(train2X, train2Y)
    Chart.Point(test2X, est2Y)
]

<!-- Plotly chart will be drawn inside this DIV -->